In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Agents for Engineers (Evolution of AI Agents)

<a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/workshops/ai-agents/ai_agents_for_engineers.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

| | |
|-|-|
| Author(s) | [Kristopher Overholt](https://github.com/koverholt) [Holt Skinner](https://github.com/holtskinner)|

## Overview

This notebook demonstrates 3 different approaches to generating essays using the [Gemini Developer API](https://ai.google.dev/gemini-api/docs) or [Gemini API in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs). Each method illustrates a distinct paradigm for running AI Agents in differing levels of complexity.

1. Zero-Shot Approach with the Gemini API
2. Step-by-Step Approach With LangChain
3. Iterative, AI-Agent Approach with LangGraph

## Get started

### Install Gemini SDK and other required packages


In [ ]:
%pip install --upgrade --quiet \
    google-genai \
    langgraph \
    langchain \
    langchain-google-genai \
    langchain-google-vertexai \
    langchain-community \
    tavily-python \
    pydantic

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Configure Tavily

Get an API key for [Tavily](https://tavily.com/), a web search API for Generative AI models.

In [2]:
import os

os.environ["TAVILY_API_KEY"] = "YOUR_API_KEY_HERE"

In [ ]:
# If your API Keys are in Colab Secrets
import sys

if "google.colab" in sys.modules:
    from google.colab import userdata

    os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

### Configure Gemini Developer API

Get API keys from [Google AI Studio](https://ai.google.dev/gemini-api/docs/api-key) and [Tavily](https://tavily.com/).

In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE"

In [ ]:
# If your API Keys are in Colab Secrets
import sys

if "google.colab" in sys.modules:
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

### Configure Vertex AI

**Use a Google Cloud Project:** This requires enabling the Vertex AI API in your Google Cloud project.

[Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [4]:
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "true"

## Generating Essays Using a Zero-Shot Approach with the Gemini API

With just a single call to the `generate_content` method, users can create detailed, structured essays on any topic by leveraging state-of-the-art language models such as Gemini 2.0 or Gemini 2.0.

<img src="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/ai-agents/1-prompt-essay.png?raw=1" width="350px">

### Import libraries

In [5]:
from IPython.display import Markdown, display

### Create Client

In [6]:
from google import genai

client = genai.Client()

Verify which API you are using.

In [ ]:
if not client.vertexai:
    print(f"Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )
elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

### Load model

In [8]:
MODEL_ID = "gemini-2.0-flash-001"

### Make an API call to generate the essay

In [9]:
prompt = "Write a 3-paragraph essay about the application of heat transfer in modern data centers"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

Heat transfer is a critical consideration in the design and operation of modern data centers, as the concentration of high-powered computing equipment generates tremendous amounts of heat. Effective heat management is essential for maintaining optimal performance, preventing equipment failure, and ensuring energy efficiency. Various heat transfer mechanisms, including conduction, convection, and radiation, are leveraged in innovative cooling solutions. For instance, cold plates directly attached to CPUs and GPUs conduct heat away, while air and liquid cooling systems rely on convection to dissipate heat from the components to the surrounding environment. The use of phase-change materials and heat pipes further enhances heat transfer efficiency, enabling higher densities and improved reliability in data center operations.

Modern data centers employ a range of sophisticated cooling technologies to manage the generated heat effectively. Air cooling, once the dominant method, is now often augmented with liquid cooling solutions, particularly for high-density deployments. Direct-to-chip liquid cooling offers superior thermal performance compared to air cooling, allowing for higher clock speeds and reduced fan noise. Rear door heat exchangers and immersion cooling systems further enhance heat dissipation by capturing heat directly from the exhaust air or submerging the equipment in a dielectric fluid. Computational fluid dynamics (CFD) simulations are used to model airflow and temperature distribution within the data center, enabling optimized placement of cooling units and servers for efficient heat removal.

The implementation of advanced heat transfer technologies in data centers not only improves equipment reliability and performance but also contributes to significant energy savings and reduced environmental impact. By optimizing cooling systems, data centers can minimize their power consumption, lowering operating costs and decreasing their carbon footprint. Furthermore, waste heat recovery systems can capture and repurpose the excess heat generated by the servers, utilizing it for building heating or other applications, further improving energy efficiency and sustainability. As data centers continue to evolve and house increasingly powerful computing hardware, the effective management of heat transfer will remain a crucial challenge and a key enabler of their continued growth and success.


---

However, what if we ask the model to write an essay about an event that happened more recently and the LLM doesn't inherently know about that event?

In [10]:
prompt = "Write a 3-paragraph essay about the impacts of Hurricane Helene and Hurricane Milton in 2024."
response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

While specific details regarding the impact of hypothetical 2024 hurricanes Helene and Milton are impossible to predict without them actually occurring, we can discuss the *potential* ramifications based on hurricane characteristics and historical precedent. A major impact would undoubtedly be widespread damage to infrastructure, particularly in coastal communities. Strong winds could topple power lines, leaving residents without electricity for extended periods, while torrential rainfall could lead to catastrophic flooding, damaging homes, businesses, and critical transportation networks. Depending on the intensity and track, these storms could trigger evacuations, straining local resources and potentially causing displacement of entire populations. Moreover, the economic repercussions would be significant, affecting tourism, agriculture, and the fishing industry.

Beyond the immediate physical damage, both hurricanes could have devastating effects on human lives and well-being. Injuries and fatalities are always a concern, particularly amongst vulnerable populations with limited access to resources or residing in flood-prone areas. The stress and trauma associated with experiencing a natural disaster can lead to long-term mental health issues, requiring extensive support services. Disruption to healthcare facilities and emergency services could further exacerbate the crisis, hindering rescue and recovery efforts. In the aftermath, access to clean water, food, and shelter would become paramount concerns, demanding a coordinated response from governmental and non-governmental organizations.

Looking at the broader picture, Hurricanes Helene and Milton, even if categorized as less intense storms, could significantly impact the environment. Coastal erosion and beach degradation would be likely, altering shorelines and potentially endangering fragile ecosystems. Runoff from floodwaters could contaminate water supplies and spread pollutants, impacting both human and marine life. The destruction of habitats could lead to displacement of wildlife and disruption of ecological balance. Ultimately, the scale of the impact would depend on a confluence of factors, including the storms' intensity, track, and the preparedness levels of affected communities, highlighting the crucial need for robust disaster preparedness and mitigation strategies.


In this case, the model had no information about these recent events and was unable to write an effective essay.

## Generating Essays Using a Step-by-Step Approach With LangChain

This step demonstrates how to build an essay-writing pipeline using [LangChain](https://www.langchain.com/), the [Gemini API in Google AI Studio](https://ai.google.dev/gemini-api/docs), and [Tavily](https://tavily.com/) for search.

By combining these tools, we create a seamless workflow that plans an essay outline, performs web searches for relevant information, and generates a complete essay draft based on the collected data.

This solution showcases the power of chaining LLM models and external tools to tackle complex tasks with minimal human intervention, providing a robust approach to automated content generation.

<img src="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/ai-agents/2-langchain-essay.png?raw=1" width="550px">


### Import libraries

In [14]:
from IPython.display import Markdown, display
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI

### Initialize Gemini model & search tool

In [15]:
if client.vertexai:
    model = ChatVertexAI(project=PROJECT_ID, model=MODEL_ID, temperature=0)
else:
    model = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=0)
tavily_tool = TavilySearchResults(max_results=5)

### Define prompt templates and Runnables

In [16]:
# Planning: Create an outline for the essay
outline_template = ChatPromptTemplate.from_template(
    "Create a detailed outline for an essay on {topic}"
)


# Research: Web search
def research_fn(topic):
    response = tavily_tool.invoke({"query": topic})
    return "\n".join([f"- {result['content']}" for result in response])


# Writing: Write the essay based on outline and research
writing_template = ChatPromptTemplate.from_template(
    "Based on the following outline and research, write a 3-paragraph essay on '{topic}':\n\nOutline:\n{outline}\n\nResearch:\n{research}\n\nEssay:"
)

### Define the Runnable Chain using [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel)

In [ ]:
# Define individual chains
outline_chain = LLMChain(llm=model, prompt=outline_template)
writing_chain = LLMChain(llm=model, prompt=writing_template)

# Use the pipe operator to combine chains
chain = (
    outline_chain
    | (
        lambda result: {
            "topic": result["topic"],
            "outline": result["text"],
            "research": research_fn(result["topic"]),
        }
    )
    | writing_chain
    | (lambda result: result["text"])  # Extract the essay text from the final result
    | StrOutputParser()
)

### Generate the essay

In [18]:
essay = chain.invoke({"topic": prompt})
display(Markdown(essay))

As climate change continues to fuel warmer ocean temperatures, the threat of increasingly powerful and frequent hurricanes looms large. Understanding the potential impacts of these storms is crucial for effective preparation and mitigation efforts. In late September and early October of 2024, Florida was struck by two such storms: Hurricane Helene, a Category 3 storm, and Hurricane Milton, a Category 1 storm. While both hurricanes brought destruction, their differing intensities and paths resulted in distinct impacts, ranging from widespread wastewater contamination to significant infrastructural damage and economic disruption.

Hurricane Helene, the stronger of the two, caused significant environmental damage in Florida. St. Petersburg's infrastructure was overwhelmed, leading to the discharge of approximately 1.5 million gallons of untreated wastewater into the surrounding areas. The Glendale Wastewater Reclamation Facility in Lakeland, operating since 1926, experienced two major spills totaling five million gallons. This contamination posed a serious threat to public health and the delicate ecosystems of the Tampa Bay area. The coastal region experienced increased rainfall, which climate models indicated was increased in both likelihood and intensity.

In contrast, Hurricane Milton, while weaker, inflicted substantial economic and infrastructural damage across Florida. Making landfall near Siesta Key as a Category 3 storm on October 9, 2024, Milton disrupted key industries and essential services. The fishing industry in Apalachicola Bay suffered greatly, with damage to boats, docks, and processing facilities leading to significant income loss for local fishermen. Debris removal and cleanup efforts were extensive, and the combined impact of Helene and Milton contributed to the ten costliest climate disasters of 2024, which caused more than $229 billion in damages globally. World Vision, through its 20 church partnerships, served over 39,600 people in Florida, Georgia, and the Carolinas in response to the hurricanes.

While both hypothetical hurricanes brought destruction, Helene's impact was primarily focused on environmental contamination, while Milton caused significant economic disruption and infrastructure damage. As the threat of increasingly intense hurricanes continues to grow, proactive measures to enhance preparedness, strengthen infrastructure, and foster community resilience are essential to minimize the devastating impacts of these powerful storms. The events of late September and early October 2024 serve as a stark reminder of the urgent need for continued research, investment in resilient infrastructure, and community-based preparedness initiatives to mitigate the impacts of future hurricanes.


## Generating Essays Using an Iterative, AI-Agent Approach with LangGraph

This section demonstrates how to build a [LangGraph](https://langchain-ai.github.io/langgraph/)-powered AI agent to generate, revise, and critique essays using large language models such as Google's [Gemini API in Google AI Studio](https://ai.google.dev/gemini-api/docs) or the [Gemini API in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/overview). The LangGraph code was adapted from the awesome DeepLearning.AI course on [AI Agents in LangGraph](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/).

By defining a structured state flow with nodes such as "Planner," "Research Plan," "Generate," "Reflect," and "Research Critique," the system iteratively creates an essay on a given topic, incorporates feedback, and provides research-backed insights.

<img src="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/ai-agents/3-langgraph-essay.png?raw=1" width="900px">

The workflow enables automated essay generation with revision controls, making it ideal for structured writing tasks or educational use cases. Additionally, the notebook uses external search tools to gather and integrate real-time information into the essay content.

### Import libraries

In [ ]:
from typing import TypedDict

# Common libraries
from IPython.display import Image, Markdown, display

# LangChain and LangGraph components
from langchain_core.messages import HumanMessage, SystemMessage

# LangChain integrations for Gemini API in Google AI Studio and Vertex AI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph

# Typing utilities for data validation and schema definitions
from pydantic.v1 import BaseModel

# Tavily client for performing web searches
from tavily import TavilyClient

### Initialize agent memory, agent state, and schema for search queries

In [20]:
# Initialize agent memory
memory = MemorySaver()


# Define the agent's state
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: list[str]
    revision_number: int
    max_revisions: int


# Define a schema for search queries
class Queries(BaseModel):
    """Variants of query to search for"""

    queries: list[str]

### Initialize Gemini model and search tool

Remember to set the environment variables `GOOGLE_API_KEY` and `TAVILY_API_KEY`. And configure credentials for Vertex AI if you switch to it.

In [21]:
if client.vertexai:
    model = ChatVertexAI(project=PROJECT_ID, model=MODEL_ID, temperature=0)
else:
    model = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=0)

# Initialize Tavily client for performing web searches
tavily = TavilyClient()

### Define prompt templates for each stage

In [22]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay.
Write such an outline for the user provided topic. Give an outline of the essay along with any
relevant notes or instructions for the sections."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 3-paragraph essays.
Generate the best essay possible for the user's request and the initial outline.
If the user provides critique, respond with a revised version of your previous attempts.
Use Markdown formatting to specify a title and section headers for each paragraph.
Utilize all of the information below as needed:
---
{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission.
Generate critique and recommendations for the user's submission.
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can
be used when writing the following essay. Generate a list of search queries that will gather
any relevant information. Only generate 3 queries max."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can
be used when making any requested revisions (as outlined below).
Generate a list of search queries that will gather any relevant information.
Only generate 3 queries max."""

### Define node functions for each stage

In [23]:
# Generate an outline for the essay


def plan_node(state: AgentState):
    messages = [SystemMessage(content=PLAN_PROMPT), HumanMessage(content=state["task"])]
    response = model.invoke(messages)
    return {"plan": response.content}


# Conducts research based on the generated plan and web search results
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke(
        [
            SystemMessage(content=RESEARCH_PLAN_PROMPT),
            HumanMessage(content=state["task"]),
        ]
    )
    content = state["content"] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}


# Generates a draft based on the content and plan
def generation_node(state: AgentState):
    content = "\n\n".join(state["content"] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}"
    )
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message,
    ]
    response = model.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1,
    }


# Provides feedback or critique on the draft
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state["draft"]),
    ]
    response = model.invoke(messages)
    return {"critique": response.content}


# Conducts research based on the critique
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke(
        [
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
            HumanMessage(content=state["critique"]),
        ]
    )
    content = state["content"] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}


# Determines whether the critique and research cycle should
# continue based on the number of revisions
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

### Define and compile the graph

In [24]:
# Initialize the state graph
builder = StateGraph(AgentState)

# Add nodes for each step in the workflow
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

# Set the entry point of the workflow
builder.set_entry_point("planner")

# Add conditional edges for task continuation or end
builder.add_conditional_edges(
    "generate", should_continue, {END: END, "reflect": "reflect"}
)

# Define task sequence edges
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

# Compile the graph with memory state management
graph = builder.compile(checkpointer=memory)

### Show the compiled graph

In [ ]:
Image(graph.get_graph().draw_mermaid_png())

### Run the agent - write on!

In [30]:
# Define the topic of the essay
ESSAY_TOPIC = "What were the impacts of Hurricane Helene and Hurricane Milton in 2024?"

# Define a thread configuration with a unique thread ID
thread = {"configurable": {"thread_id": "1"}}

# Stream through the graph execution with an initial task and state
for s in graph.stream(
    {
        "task": ESSAY_TOPIC,  # Initial task
        "max_revisions": 2,  # Maximum number of revisions allowed
        "revision_number": 1,  # Current revision number
        "content": [],  # Initial empty content list
    },
    thread,
):
    step = next(iter(s))
    display(Markdown(f"# {step}"))
    for key, content in s[step].items():
        if key == "revision_number":
            display(Markdown(f"**Revision Number**: {content}"))
        elif isinstance(content, list):
            for c in content:
                display(Markdown(c))
        else:
            display(Markdown(content))
    print("\n---\n")

# planner

Okay, here's a high-level outline for an essay exploring the impacts of Hurricanes Helene and Milton in 2024. This outline focuses on providing a structure for a comprehensive analysis, considering various sectors and potential long-term consequences.

**Essay Title (Example):**  Twin Storms, Lasting Scars: A Comparative Analysis of the Impacts of Hurricanes Helene and Milton (2024)

**I. Introduction**

*   **Hook:** Start with a compelling statistic or anecdote about the 2024 hurricane season or the general increase in extreme weather events.
*   **Background:** Briefly introduce Hurricane Helene and Hurricane Milton, including their formation, paths, and peak intensities. Mention the geographical areas most affected by each storm.
*   **Thesis Statement:**  Clearly state the essay's main argument.  For example: "While both Hurricanes Helene and Milton caused significant damage in 2024, their impacts differed substantially due to variations in intensity, geographical location, and pre-existing vulnerabilities, resulting in distinct challenges for affected communities in terms of infrastructure, economy, and public health."
*   **Roadmap:** Briefly outline the key areas the essay will cover (e.g., infrastructure damage, economic consequences, environmental effects, and social impacts).

**II. Hurricane Helene: Impacts and Analysis**

*   **A. Geographical Context and Vulnerabilities:**
    *   Describe the specific regions affected by Hurricane Helene.
    *   Analyze the pre-existing vulnerabilities of these regions (e.g., coastal erosion, poverty, inadequate infrastructure, reliance on specific industries).
    *   Discuss any evacuation efforts and their effectiveness.
*   **B. Infrastructure Damage:**
    *   Detail the damage to critical infrastructure: power grids, transportation networks (roads, bridges, ports), communication systems, water and sanitation facilities.
    *   Provide specific examples and, if available, estimated costs of repair.
    *   Discuss the immediate and long-term consequences of infrastructure failure (e.g., power outages, disruptions to supply chains).
*   **C. Economic Consequences:**
    *   Analyze the impact on key industries in the affected regions (e.g., tourism, agriculture, fishing).
    *   Discuss job losses, business closures, and disruptions to trade.
    *   Examine the role of insurance and disaster relief in mitigating economic losses.
*   **D. Environmental Effects:**
    *   Assess the environmental damage caused by Helene: coastal erosion, flooding, saltwater intrusion, damage to ecosystems (e.g., wetlands, coral reefs).
    *   Discuss the potential long-term ecological consequences.
*   **E. Social Impacts:**
    *   Examine the impact on public health: injuries, disease outbreaks, mental health issues.
    *   Discuss displacement, homelessness, and food insecurity.
    *   Analyze the role of community organizations and government agencies in providing aid and support.

**III. Hurricane Milton: Impacts and Analysis**

*   **A. Geographical Context and Vulnerabilities:**
    *   Describe the specific regions affected by Hurricane Milton.
    *   Analyze the pre-existing vulnerabilities of these regions (e.g., different demographics, building codes, economic activities compared to Helene's affected areas).
    *   Discuss any evacuation efforts and their effectiveness.
*   **B. Infrastructure Damage:**
    *   Detail the damage to critical infrastructure, noting any differences from the types of damage caused by Helene.
    *   Provide specific examples and estimated costs of repair.
    *   Discuss the immediate and long-term consequences of infrastructure failure.
*   **C. Economic Consequences:**
    *   Analyze the impact on key industries in the affected regions, noting any differences from Helene's affected areas.
    *   Discuss job losses, business closures, and disruptions to trade.
    *   Examine the role of insurance and disaster relief in mitigating economic losses.
*   **D. Environmental Effects:**
    *   Assess the environmental damage caused by Milton, noting any differences from Helene.
    *   Discuss the potential long-term ecological consequences.
*   **E. Social Impacts:**
    *   Examine the impact on public health, noting any differences from Helene's affected areas.
    *   Discuss displacement, homelessness, and food insecurity.
    *   Analyze the role of community organizations and government agencies in providing aid and support.

**IV. Comparative Analysis: Helene vs. Milton**

*   **A. Key Differences in Impacts:**
    *   Directly compare and contrast the impacts of the two hurricanes across the categories discussed above (infrastructure, economy, environment, social).
    *   Highlight the factors that contributed to these differences (e.g., storm intensity, geographical characteristics, preparedness levels).
*   **B. Lessons Learned:**
    *   Identify key lessons learned from the responses to both hurricanes.
    *   Discuss the effectiveness of different mitigation and adaptation strategies.
    *   Consider the implications for future hurricane preparedness and disaster management.

**V. Conclusion**

*   **Restate Thesis:** Summarize the main argument of the essay, emphasizing the distinct impacts of the two hurricanes.
*   **Broader Implications:** Discuss the broader implications of these events in the context of climate change and increasing hurricane intensity.
*   **Call to Action (Optional):** Suggest potential actions that can be taken to improve hurricane preparedness, resilience, and disaster response in the future.
*   **Concluding Thought:** End with a powerful statement about the challenges posed by hurricanes and the importance of proactive measures to protect vulnerable communities.

**Important Notes:**

*   **Sources:**  Use credible sources, including government reports (e.g., NOAA, FEMA), academic studies, news articles from reputable sources, and reports from NGOs involved in disaster relief.
*   **Data:**  Include specific data and statistics to support your claims (e.g., rainfall amounts, wind speeds, damage estimates, number of people displaced).
*   **Visuals:** Consider including maps, charts, and photographs to enhance the essay.
*   **Objectivity:**  Present a balanced and objective analysis, avoiding sensationalism or biased reporting.
*   **Future Projections:** While focusing on the 2024 events, briefly touch upon how climate change might influence future hurricane seasons and the need for long-term adaptation strategies.
*   **Specificity:** The more specific you can be about the locations affected and the types of damage, the stronger your essay will be.

This outline provides a solid framework. Remember to adapt it based on the specific information available about Hurricanes Helene and Milton in 2024. Good luck!



---



# research_plan

North Carolina was severely impacted by Hurricane Helene during late September 2024, primarily in its western Appalachian region, causing at least 105 reported deaths and significant destruction of infrastructure and residential areas across several settlements.[1] After making landfall in the Big Bend region of Florida on September 27, the hurricane began to traverse over land across Georgia as a Category 2 hurricane and into the Appalachian mountain range as a tropical storm, depositing record-breaking amounts of rainfall across several settlements in western North Carolina, such as Asheville, Swannanoa, Spruce Pine, Chimney Rock, Montreat, Lake Lure, and several others. Of the 241 people reported victims of Hurricane Helene in the United States, 119 people were in North Carolina, surpassing the state record of 80 deaths recorded in a 1916 flood.[8][35] Of these, 72 residents of Buncombe County were among the deceased.

...


---



# generate

Okay, I will use your outline to generate a 3-paragraph essay.

**Title: The Destructive Wake of Hurricanes Helene and Milton in 2024**

**Paragraph 1: Introduction to the 2024 Hurricane Season**

The 2024 hurricane season proved to be a stark reminder of the increasing intensity and devastating potential of tropical cyclones, with Hurricanes Helene and Milton leaving a trail of destruction across the Southeastern United States. Helene, a large and catastrophic storm, made landfall in Florida before cutting a deadly path through Georgia and into the Appalachian region, while Milton impacted the Treasure Coast of Florida. Although both storms brought widespread damage, their impacts differed significantly due to variations in their tracks, intensities, and the vulnerabilities of the affected regions. This essay will explore the distinct consequences of Hurricanes Helene and Milton, highlighting the challenges faced by impacted communities in terms of infrastructure, economic stability, and overall resilience.

**Paragraph 2: Contrasting Impacts of Helene and Milton**

Hurricane Helene's impact was most acutely felt in the Appalachian region of North Carolina, where record-breaking rainfall led to catastrophic flooding and landslides. The storm claimed over 100 lives in the state and caused extensive damage to infrastructure, isolating communities and disrupting essential services. In contrast, Hurricane Milton, while less deadly, spawned a series of tornadoes in Florida, causing significant structural damage and disrupting air travel. The economic consequences also varied, with Helene severely impacting tourism and agriculture in the Appalachian region, while Milton primarily affected coastal communities and infrastructure in Florida. The differing impacts underscore the importance of considering the specific vulnerabilities of each region when preparing for and responding to hurricanes.

**Paragraph 3: Lessons Learned and the Future of Hurricane Preparedness**

The experiences of Hurricanes Helene and Milton in 2024 offer valuable lessons for future hurricane preparedness and disaster management. The need for resilient infrastructure, effective evacuation plans, and coordinated disaster relief efforts was evident in both cases. Furthermore, the increasing intensity of these storms highlights the urgent need to address climate change and its contribution to extreme weather events. By understanding the distinct impacts of these hurricanes and investing in proactive measures, communities can better protect themselves from the devastating consequences of future storms.


**Revision Number**: 3


---



### Output the final draft of the essay

In [31]:
display(Markdown(s["generate"]["draft"]))

Okay, I will use your outline to generate a 3-paragraph essay.

**Title: The Destructive Wake of Hurricanes Helene and Milton in 2024**

**Paragraph 1: Introduction to the 2024 Hurricane Season**

The 2024 hurricane season proved to be a stark reminder of the increasing intensity and devastating potential of tropical cyclones, with Hurricanes Helene and Milton leaving a trail of destruction across the Southeastern United States. Helene, a large and catastrophic storm, made landfall in Florida before cutting a deadly path through Georgia and into the Appalachian region, while Milton impacted the Treasure Coast of Florida. Although both storms brought widespread damage, their impacts differed significantly due to variations in their tracks, intensities, and the vulnerabilities of the affected regions. This essay will explore the distinct consequences of Hurricanes Helene and Milton, highlighting the challenges faced by impacted communities in terms of infrastructure, economic stability, and overall resilience.

**Paragraph 2: Contrasting Impacts of Helene and Milton**

Hurricane Helene's impact was most acutely felt in the Appalachian region of North Carolina, where record-breaking rainfall led to catastrophic flooding and landslides. The storm claimed over 100 lives in the state and caused extensive damage to infrastructure, isolating communities and disrupting essential services. In contrast, Hurricane Milton, while less deadly, spawned a series of tornadoes in Florida, causing significant structural damage and disrupting air travel. The economic consequences also varied, with Helene severely impacting tourism and agriculture in the Appalachian region, while Milton primarily affected coastal communities and infrastructure in Florida. The differing impacts underscore the importance of considering the specific vulnerabilities of each region when preparing for and responding to hurricanes.

**Paragraph 3: Lessons Learned and the Future of Hurricane Preparedness**

The experiences of Hurricanes Helene and Milton in 2024 offer valuable lessons for future hurricane preparedness and disaster management. The need for resilient infrastructure, effective evacuation plans, and coordinated disaster relief efforts was evident in both cases. Furthermore, the increasing intensity of these storms highlights the urgent need to address climate change and its contribution to extreme weather events. By understanding the distinct impacts of these hurricanes and investing in proactive measures, communities can better protect themselves from the devastating consequences of future storms.


## Additional Resources

- [Google Cloud Generative AI repository on GitHub](https://github.com/GoogleCloudPlatform/generative-ai/)
- [Gemini API in Google AI Studio](https://ai.google.dev/gemini-api/docs)
- [Gemini API in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/overview)
- [LangGraph tutorials](https://langchain-ai.github.io/langgraph/tutorials/)
- [DeepLearning.AI course on AI Agents in LangGraph](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/)